In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, Pool


df_model = pd.read_csv('/home/utilisateur/Documents/dev_ia/US_SBA/loan_project/SBAnational.csv')


In [ ]:

df_model = df_model.drop(['Name','City','Bank','CreateJob','ApprovalDate','RetainedJob','ChgOffDate', 'DisbursementDate','DisbursementGross','BalanceGross','ChgOffPrinGr'],axis=1)
df_model

### Nettoyage des colonnes ApprovalFY GrAppv SBA_Appv LowDoc MIS_Status FranchiseCode

In [3]:
# Elimine les erreurs qui ne sont pas des INT du type '1997A'
df_model['ApprovalFY'] = pd.to_numeric(df_model['ApprovalFY'], errors='coerce').astype('Int64')

# Retirer les dollars
df_model['GrAppv'] = pd.to_numeric(df_model['GrAppv'].str.replace('[\$,]', '', regex=True))
df_model['SBA_Appv'] = pd.to_numeric(df_model['SBA_Appv'].str.replace('[\$,]', '', regex=True))


# Remplace les valeurs étranges de 'RevLineCr' et 'LowDoc' par des Y et N
df_model['RevLineCr'] = df_model['RevLineCr'].replace(('T','`',',','C','3','2','R','7','A','5','.','4','-','Q'), 'IdK')
df_model['RevLineCr'] = df_model['RevLineCr'].replace(('0','1'), ('N','Y'))

# Si le montant du prêt est inférieur à 150 000$, il s'agit d'un LowDoc
condition_lowdoc = (df_model['GrAppv'] > 150000)
df_model.loc[condition_lowdoc, 'LowDoc'] = 'N'
df_model.loc[~condition_lowdoc, 'LowDoc'] = 'Y'

# Encode la target en binaire 
df_model['MIS_Status'] = df_model['MIS_Status'].replace(('CHGOFF','P I F'), ('BAD','GOOD'))

# Encode la colonne 'FranchiseCode' en binaire, afin de séparer les franchisés et les non franchisés
condition_franchise = (df_model['FranchiseCode'] > 1)
df_model.loc[condition_franchise, 'FranchiseCode'] = 1
df_model.loc[~condition_franchise, 'FranchiseCode'] = 0


# Imputation
imputer = SimpleImputer(strategy='most_frequent')
df_model[['LowDoc']] = imputer.fit_transform(df_model[['LowDoc']])

imputer = SimpleImputer(strategy='most_frequent')
df_model[['ApprovalFY']] = imputer.fit_transform(df_model[['ApprovalFY']])

imputer = SimpleImputer(strategy='most_frequent')
df_model[['State','BankState']] = imputer.fit_transform(df_model[['State','BankState']])
df_model.isnull().sum()

df_model['ApprovalFY'] = df_model['ApprovalFY'].astype(int)
df_model['GrAppv'] = df_model['GrAppv'].astype(int)
df_model['SBA_Appv'] = df_model['SBA_Appv'].astype(int)





### Nettoyage de State et ZIP 

In [ ]:
# Nettoyage de Farid

# nettoyage de la colonne State

missin_state_rows = df_model[df_model['State'].isnull()]
print(missin_state_rows[['Zip', 'State']])
print()

zip_values = missin_state_rows['Zip'].tolist()

print()
state_values_for_zip = []
for zip_value in zip_values:
    city_values_for_zip = df_model[df_model['Zip'] == zip_value]['State'].unique()
    state_values_for_zip.append(city_values_for_zip)

for i, zip_value in enumerate(zip_values):
    print(f"pour le {zip_value},  {state_values_for_zip[i]}")
print()


zip_state_pairs = [(8070,'NJ'),
                   (95682,'CA'),
                   (67219,'KS'),
                   (79925,'TX'),
                   (33410,'FL'),
                   (54205,'WI'),
                   (54025, 'MN'),
                   (84124,'UT'),
                   (65049,'MO'),
                   (75236,'TX'),
                   (76052,'TX'),
                   (76645,'TX'),

]

for zip_value, state_value in zip_state_pairs:
    df_model.loc[(df_model['Zip']== zip_value) & (df_model['State'].isnull()), 'State'] = state_value


# print(city_values_for_zip)

print()

# Traitement quand Zip == 0

# Remplacer les valeurs 0 par NaN pour faciliter le traitement
df_model['Zip'] = df_model['Zip'].replace(0, pd.NA)

# State quand Zip is Nan 
state_values_with_zip_0 = df_model.query('Zip.isna()')['State'].unique()
print(state_values_with_zip_0)

# Boucle pour itérer sur chaque état dans la liste
for state in state_values_with_zip_0:
    # Utiliser query pour filtrer les lignes correspondantes à l'état actuel et extraire les codes postaux uniques
    unique_zips_for_state = df_model.query('State == @state')['Zip'].unique()
    
    # Afficher l'état et les codes postaux correspondants
    print(f"État : {state}, Codes postaux uniques : {unique_zips_for_state}")

# Déterminer le code postal le plus fréquent pour chaque état
most_frequent_zip = df_model.groupby('State')['Zip'].transform(lambda x: x.mode()[0])

# Remplacer les valeurs NaN par le code postal le plus fréquent
df_model['Zip'].fillna(most_frequent_zip, inplace=True)

### Nettoyage de Bank

In [ ]:
# nettoyage Bank 

# df_model['Bank'].head(1000)

# bank_state_counts = df_model.groupby('Bank')['BankState'].nunique()
# classement_bank = bank_state_counts.sort_values(ascending=False)
# print(classement_bank)

# correlation

df_model['BankState'].unique()
contingency_table = pd.crosstab(df_model['BankState'], df_model['State'])
print(contingency_table)

# chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# print(p_value)

# Qd BankState est Nan, connaitre les valeurs de State

states_with_nan_bankstate = df_model.query('BankState.isnull()')['State'].unique()
print(states_with_nan_bankstate)

# determiné la valeur de Bankstate la plus fréquente lorsque State contient une valeur de states_with_nan_bankstate

most_common_bank_states = {}

for state in states_with_nan_bankstate:
    filtered_df = df_model[df_model['State']== state]
    most_common_bank_state = filtered_df['BankState'].mode()
    most_common_bank_states[state] = most_common_bank_state

for state, bank_state in most_common_bank_states.items():
    print(f"Pour l'état {state} la valeur la plus fréquente de BankState est {bank_state}")
    df_model.loc[(df_model['State'] == state) & (df_model['BankState'].isnull()), 'BankState'] = bank_state.values[0]

# # Déterminer l'etat de le state le plus fréquent pour chaque bank state 
# most_frequent_state = df_model.groupby('State')['BankState'].transform(lambda x: x.mode()[0])
# print(most_frequent_state)


# # Remplacer les valeurs NaN par le code postal le plus fréquent
# df_model['Zip'].fillna(most_frequent_zip, inplace=True)

### Encodage de NoEmp et NAICS

In [6]:
# Créer une colonne pour la catégorie d'entreprise en fonction du nombre d'employés
df_model['Entreprise_category'] = pd.cut(df_model['NoEmp'], 
                                          bins=[-np.inf, 10, 250, 5000, np.inf], 
                                          labels=['TPE', 'PME', 'ETI', 'TGE'], 
                                          right=False)

# Convertir la catégorie en chaîne de caractères pour la lisibilité
df_model['Entreprise_category'] = df_model['Entreprise_category'].astype(str)

# # On drop la colonne NoEmp
df_model = df_model.drop('NoEmp', axis=1)
# Dictionnaire de correspondance des codes NAICS aux catégories
df_model['NAICS_Category'] = df_model['NAICS'].astype(str).str[:2]
df_model['NAICS_Category'] = df_model['NAICS_Category'].astype(int)



naics_categories = {
    '0' : 'Inconnue',
    '11': 'Agriculture, Forestry, Fishing and Hunting',
    '21': 'Mining, Quarrying, and Oil and Gas Extraction',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale Trade',
    '44': 'Retail Trade',
    '45': 'Retail Trade',
    '48': 'Transportation and Warehousing',
    '49': 'Transportation and Warehousing',
    '51': 'Information',
    '52': 'Finance and Insurance',
    '53': 'Real Estate and Rental and Leasing',
    '54': 'Professional, Scientific, and Technical Services',
    '55': 'Management of Companies and Enterprises',
    '56': 'Administrative and Support and Waste Management and Remediation Services',
    '61': 'Educational Services',
    '62': 'Health Care and Social Assistance',
    '71': 'Arts, Entertainment, and Recreation',
    '72': 'Accommodation and Food Services',
    '81': 'Other Services (except Public Administration)',
    '92': 'Public Administration'
}

# Appliquer la correspondance à la colonne contenant les deux premiers chiffres des codes NAICS
df_model['NAICS_Category'] = df_model['NAICS'].astype(str).str[:2].map(naics_categories)

# On drop la colonne NAICS
df_model = df_model.drop('NAICS', axis=1)

### Y a t il des 0 dans le df ?

In [ ]:

colonne_avec_zero = ['LoanNr_ChkDgt','State','Zip','BankState','ApprovalFY','Term','NewExist','FranchiseCode','UrbanRural','RevLineCr','LowDoc','GrAppv','SBA_Appv','NAICS_Category']

for i in colonne_avec_zero:
    nombre_de_zeros = (df_model[i] == 0).sum()
    print(f"Nombre de zéros dans la colonne {i}:", nombre_de_zeros)
colonne_avec_zero = ['LoanNr_ChkDgt','State','Zip','BankState','ApprovalFY','Term','NewExist','FranchiseCode','UrbanRural','RevLineCr','LowDoc','GrAppv','SBA_Appv','NAICS_Category']

for i in colonne_avec_zero:
    nombre_de_zeros = (df_model[i] == 0).sum()
    print(f"Nombre de zéros dans la colonne {i}:", nombre_de_zeros)

### Drop les valeurs manquantes du df

In [8]:

df_model = df_model.dropna()


### nettoyage de NewExist

In [9]:
pd.set_option('display.max_rows', None)
imputer = SimpleImputer(strategy='most_frequent')

# Adapter l'imputeur aux données avec les valeurs différentes de 0.0
imputer.fit(df_model[df_model['NewExist'] != 0.0][['NewExist']])

# Remplacer les valeurs 0.0 par la valeur la plus fréquente
df_model['NewExist'] = imputer.transform(df_model[['NewExist']]).ravel()




### Nettoyage de Term

In [ ]:
import numpy as np

# Remplacez les zéros par NaN dans la colonne "Term"
df_model['Term'] = df_model['Term'].replace(0, np.nan)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Créer un objet IterativeImputer
imputer = IterativeImputer(random_state=0)

# Sélectionner les valeurs à imputer (valeurs NaN dans la colonne "Term")
X_impute = df_model[['Term']]

# Imputer les valeurs NaN
X_imputed = imputer.fit_transform(X_impute)

# Mettre à jour la colonne "Term" avec les valeurs imputées
df_model['Term'] = X_imputed

# Vérifier à nouveau les valeurs uniques pour confirmer les modifications
print(df_model['Term'].unique())

df_model['Term'] = df_model['Term'].astype(int)




In [11]:
colonnes_supp = ['LoanNr_ChkDgt','NewExist','FranchiseCode','UrbanRural','RevLineCr','LowDoc','Entreprise_category','NAICS_Category']
df_model = df_model.drop(colonnes_supp, axis=1)
df_model.dtypes

State         object
Zip            int64
BankState     object
ApprovalFY     int64
Term           int64
MIS_Status    object
GrAppv         int64
SBA_Appv       int64
dtype: object

In [12]:
df_model['Zip'].unique()

array([47711, 46526, 47401, ..., 70036, 66549, 26134])

In [13]:
df_model['State'] = df_model['State'].astype(str)
df_model['BankState'] = df_model['BankState'].astype(str)
df_model['Zip'] = df_model['Zip'].astype(int)
df_model['ApprovalFY'] = df_model['ApprovalFY'].astype(int)
df_model['Term'] = df_model['Term'].astype(int)
df_model['GrAppv'] = df_model['GrAppv'].astype(int)
df_model['SBA_Appv'] = df_model['SBA_Appv'].astype(int)

In [22]:
df_model.dtypes

State         object
Zip            int64
BankState     object
ApprovalFY     int64
Term           int64
MIS_Status    object
GrAppv         int64
SBA_Appv       int64
dtype: object

### modelisation catboost

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier


X = df_model.drop(['MIS_Status'], axis=1)
y = df_model.MIS_Status


X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.95, random_state=42, stratify=y)


num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

onehotscale_pipeline = make_pipeline(OneHotEncoder(handle_unknown='ignore'))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)


my_final_pipeline = make_pipeline(preprocessing)
my_final_pipeline.fit(X_train)

feature_names = my_final_pipeline.get_feature_names_out(X.columns)

model = make_pipeline(
    my_final_pipeline,
    # RandomForestClassifier(random_state=42,max_depth=50)
    CatBoostClassifier(random_state=42, depth=6,verbose=False)
)

# Grid Search

# Définissez les paramètres de la grille que vous souhaitez rechercher
param_grid = {
    'catboostclassifier__n_estimators': [100, 200, 300],
    'catboostclassifier__depth': [4, 6, 8]   
}

# Créez l'objet GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Fit du modèle sur les données d'entraînement
grid_search.fit(X_train, y_train)

# Affichage des meilleurs paramètres et de la meilleure précision
print("Meilleurs paramètres trouvés :")
print(grid_search.best_params_)

# Prédiction sur les données de test avec le meilleur modèle
y_pred = grid_search.predict(X_test) 

report = classification_report(y_test, y_pred)

print(report)


# print("Confusion Matrix:")
# display(confusion_matrix(y_test, y_pred))

# # Obtenez l'importance des caractéristiques à partir de l'attribut feature_importances_
# feature_importances = model.named_steps['catboostclassifier'].feature_importances_

# # Créer une DataFrame pour associer les caractéristiques à leurs importances
# importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

# # Trier les caractéristiques par ordre d'importance
# importance_df = importance_df.sort_values(by='Importance', ascending=False)

# # Afficher les caractéristiques les plus importantes
# print(importance_df.head(10))  # Par exemple, les 10 premières caractéristiques les plus importantes

### Sauvergarde et chargement Pickle

In [16]:
# import pickle


# # Sauvegarder le modèle dans un fichier
# with open('modele.pkl', 'wb') as fichier:
#     pickle.dump(model, fichier)

# with open('modele.pkl', 'rb') as fichier:
#     modele_charge = pickle.load(fichier)

# def prediction(modele_charge, data):
#     predictions = modele_charge.predict(data)
#     return predictions

from joblib import dump, load

model_path = 'model.pkl'
dump(model, model_path)
model = load('model.pkl')


In [17]:
model.score(X_test,y_test)

0.9529646394478555

### Test de la fonction Predict

In [31]:
def predict_prod(model,data):
    # Convertir la liste de listes en DataFrame
    df = pd.DataFrame(data, columns=['State', 'Zip', 'BankState', 'ApprovalFY', 'Term', 'GrAppv', 'SBA_Appv'])
    predictions = model.predict(df)
    return predictions

predict_prod(model,[["IN", 47711, "OH", 1997, 84, 60000, 48000]])





array(['GOOD'], dtype=object)